In [71]:
from ase import Atoms
from ase.atom import Atom
from ase.io import read, write
from ase.visualize import view # Importa la herramienta para visualizar la estructura

In [72]:
mol = read('1-Ethyl-3-methylimidazolium.xyz')

mol.cell = [[20,0.0,0.0],[0.0,20,0.0],[0.0,0.0,20]]
mol.positions[:,0] += 1
mol.positions[:,1] += 1
mol.positions[:,2] += 1
mol.center()

view(mol)

<Popen: returncode: None args: ['/home/hernan-barquero/anaconda3/envs/genera...>

In [73]:
# Posicion del hidrogeno 10
mol[10].position

array([10.4104 ,  7.9867 ,  9.50215])

In [74]:
# Posicion del Nitrogeno 0 y 1
display(mol[0].position,mol[1].position)

array([9.5687 , 9.9674 , 9.64565])

array([11.6816 ,  9.6685 ,  9.96395])

In [75]:
# Array de posiciones para el cloro
import numpy as np
cloro_pos = np.empty((6,3))

z_valor = np.mean([mol[0].position[2], mol[1].position[2]])
x_diff = np.abs(mol[0].position[0] - mol[1].position[0])
y_valor = 2.204 *3


idx = np.arange(1,cloro_pos.shape[0]+1,1)

cloro_pos[:,2] = z_valor

cloro_pos[:,0] = mol[0].position[0] + x_diff/idx
cloro_pos[:,1] = mol[10].position[1] - y_valor/idx

cloro_pos

array([[11.6816  ,  1.3747  ,  9.8048  ],
       [10.62515 ,  4.6807  ,  9.8048  ],
       [10.273   ,  5.7827  ,  9.8048  ],
       [10.096925,  6.3337  ,  9.8048  ],
       [ 9.99128 ,  6.6643  ,  9.8048  ],
       [ 9.92085 ,  6.8847  ,  9.8048  ]])

In [76]:
sistemas_cloro = []

for i in range(6):
    # Copiar el sistema original (puede ser copy.deepcopy si querés independencia completa)
    nuevo_sistema = mol.copy()

    # Crear un átomo de cloro en la posición deseada
    atomo_cl = Atom('Cl', cloro_pos[i])

    # Agregar el átomo al sistema
    nuevo_sistema.append(atomo_cl)

    # Guardar el nuevo sistema
    sistemas_cloro.append(nuevo_sistema)
    
sistemas_cloro

[Atoms(symbols='N2C6H11Cl', pbc=False, cell=[20.0, 20.0, 20.0]),
 Atoms(symbols='N2C6H11Cl', pbc=False, cell=[20.0, 20.0, 20.0]),
 Atoms(symbols='N2C6H11Cl', pbc=False, cell=[20.0, 20.0, 20.0]),
 Atoms(symbols='N2C6H11Cl', pbc=False, cell=[20.0, 20.0, 20.0]),
 Atoms(symbols='N2C6H11Cl', pbc=False, cell=[20.0, 20.0, 20.0]),
 Atoms(symbols='N2C6H11Cl', pbc=False, cell=[20.0, 20.0, 20.0])]

In [81]:
for i in range(len(sistemas_cloro)):
	# Genera un archivo de entrada para Quantum ESPRESSO con la configuración donde el hidrógeno está sobre el átomo de carbono.
	write(f"pw{i}.in", sistemas_cloro[i], format="espresso-in", 
		pseudopotentials={
		"C": "C.pbe-n-kjpaw_psl.1.0.0.UPF",  # Especifica el pseudopotencial para carbono.
		"H": "H.pbe-kjpaw_psl.1.0.0.UPF",     # Especifica el pseudopotencial para hidrógeno.
		"N": "N.pbe-n-kjpaw_psl.1.0.0.UPF",
		"Cl": "Cl.pbe-n-kjpaw_psl.1.0.0.UPF"
	}, input_data={
		"control": {
			"calculation": "relax",           # Indica que se hará una relajación de la estructura.
			"prefix": f"plusCL{i}",    # Prefijo para los archivos de salida generados por Quantum ESPRESSO.
			"outdir": ".",             # Carpeta donde se guardarán los archivos temporales de Quantum ESPRESSO.
			"pseudo_dir": "/home/ebarquero/practicas/pseudo",          # Carpeta donde se encuentran los pseudopotenciales.
			"tstress": True,
			"tprnfor": True
		},
		"system": {
			"ecutwfc": 60,                    # Corte de energía para las funciones de onda (en Ry).
			"ecutrho": 600,                   # Corte de energía para la densidad de carga (en Ry).
			"occupations": "smearing",        # Usa un método de smearing para las ocupaciones electrónicas.
			"smearing": "mp",                 # Método de smearing: Methfessel-Paxton.
			"degauss": 0.02 ,                  # Ancho del smearing en Ry.
			"ibrav":0, 							# Define una celda generica
		},
		"electrons": {
			"conv_thr": 1e-6                 # Criterio de convergencia para las iteraciones electrónicas.
		}
		
		}
	, kpts = (1,1,1),koffset = (0,0,0))

In [79]:
molecule = read('pw.out')
write('1-Ethyl-3-methylimidazolium_chloride_opt.xyz', molecule)